In [3]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import re
import random


In [4]:
# Configuración del WebDriver
options = uc.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-gpu')

driver = uc.Chrome(options=options)

try:
    # Navegar a la página de IMDb Top 250
    driver.get('https://www.imdb.com/chart/top/?ref_=nv_mv_250')
    
    # Esperar manualmente para dar tiempo a que la página se cargue
    time.sleep(random.uniform(8, 12))
    
    # Aumentar el tiempo de espera a 20 segundos
    wait = WebDriverWait(driver, 20)
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'h3.ipc-title__text')))
    
    # Obtener los títulos de las películas
    titles = driver.find_elements(By.CSS_SELECTOR, 'h3.ipc-title__text')
    
    # Extraer los títulos
    movie_titles = [title.text for title in titles]
    
    # Crear el DataFrame
    df1 = pd.DataFrame(movie_titles, columns=['peliculas'])
    
    # Esperar un intervalo aleatorio antes de finalizar
    time.sleep(random.uniform(5, 10))
    
except Exception as e:
    # Capturar captura de pantalla
    driver.save_screenshot('error_screenshot.png')
    
    # Imprimir el HTML de la página
    print(driver.page_source)
    
    # Imprimir el error
    print(e)
    
finally:
    # Cerrar el navegador
    driver.quit()

In [5]:
# Selecciona las primeras 250 filas
df1 = df1.iloc[:251]

In [6]:
# Eliminar la fila con el índice 0
df1 = df1.drop(0).reset_index(drop=True)

df1

,peliculas
0,1. Cadena perpetua
1,2. El padrino
2,3. El caballero oscuro
3,4. El padrino parte II
4,5. 12 hombres sin piedad
...,...
245,246. La leyenda del indomable
246,247. Criadas y señoras
247,248. Sucedió una noche
248,249. Aladdín


In [8]:
# Configuración del WebDriver
options = uc.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-gpu')

driver = uc.Chrome(options=options)

try:
    # Cargar la página
    url = "https://www.imdb.com/chart/top/?ref_=nv_mv_250"
    driver.get(url)
    
    # Esperar manualmente para dar tiempo a que la página se cargue
    time.sleep(random.uniform(8, 12))
    
    # Esperar a que los elementos se carguen
    wait = WebDriverWait(driver, 20)
    elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "span.sc-b189961a-8.hCbzGp.cli-title-metadata-item")))
    
    # Extraer los datos
    data = [element.text for element in elements]
    
    # Esperar un intervalo aleatorio antes de finalizar
    time.sleep(random.uniform(5, 10))
    
except Exception as e:
    # Capturar captura de pantalla
    driver.save_screenshot('error_screenshot.png')
    
    # Imprimir el HTML de la página
    print(driver.page_source)
    
    # Imprimir el error
    print(e)
    
finally:
    # Cerrar el navegador
    driver.quit()

# Crear el DataFrame
df2 = pd.DataFrame({'Metadata': data})

# Mostrar las primeras filas del DataFrame
print(df2.head())

  Metadata
0     1994
1   2h 22m
2       13
3     1972
4   2h 55m


In [9]:
df2

,Metadata
0,1994
1,2h 22m
2,13
3,1972
4,2h 55m
...,...
743,1h 30m
744,A
745,1984
746,2h 25m


In [10]:
# Definir los valores únicos para cada categoría
años_unicos = ['1994', '1972', '2008', '1974', '1957', '1993', '2003', '2001', '1966', '2002', '1999', '2010', '1980', '1990', '1975', '2014', '1995', '1946', '1954', '1991', '1998', '1997', '1977', '1985', '2019', '1960', '2000', '2023', '2024', '2006', '1988', '1962', '1942', '2011', '1936', '1968', '1979', '1931', '2012', '1981', '1950', '2018', '1940', '1986', '2009', '2017', '1984', '1964', '2016', '1963', '1952', '1983', '2004', '1992', '1959', '1941', '1944', '1958', '1987', '1971', '1973', '1989', '2007', '1927', '1948', '2020', '1976', '2005', '1965', '2013', '1921', '1961', '2022', '1982', '1939', '2015', '1996', '2021', '1925', '1978', '1926', '1924', '1953', '1949', '1928']

valoraciones_unicas = ['13', '18', 'A', '12', '14', '16', '7', 'PG-13', '(Banned)', 'PG', 'Approved', 'A/i', 'T', '7/i', 'Not Rated']

def classify_entry(entry):
    if entry in años_unicos:
        return 'Año'
    elif re.match(r'\d+h\s*\d*m|\d+m|Apta para mayores', entry):
        return 'Duración'
    elif entry in valoraciones_unicas or re.match(r'\d+', entry):
        return 'Valoración'
    else:
        return 'Desconocido'

# Aplicar la clasificación
df2['Tipo'] = df2['Metadata'].apply(classify_entry)

# Crear el nuevo DataFrame
df_final = pd.DataFrame({
    'Año': df2[df2['Tipo'] == 'Año']['Metadata'].reset_index(drop=True),
    'Duración': df2[df2['Tipo'] == 'Duración']['Metadata'].reset_index(drop=True),
    'Valoración': df2[df2['Tipo'] == 'Valoración']['Metadata'].reset_index(drop=True)
})

In [11]:
df_final

,Año,Duración,Valoración
0,1994,2h 22m,13
1,1972,2h 55m,18
2,2008,2h 32m,13
3,1974,3h 22m,18
4,1957,1h 36m,A
...,...,...,...
250,NaN,NaN,A
251,NaN,NaN,1934
252,NaN,NaN,A
253,NaN,NaN,A


In [12]:
# Extraer los títulos de df1
df1['titulo'] = df1['peliculas'].str.extract(r'\d+\.\s(.+)$', expand=False)

# Limpiar el DataFrame (opcional, para quitar filas innecesarias)
df1 = df1.dropna().reset_index(drop=True)

# Crear una lista con los títulos de las películas
titulos = df1['titulo'].tolist()

# Asegurarse de que df_final tenga el mismo número de filas que los títulos
if len(df_final) == len(titulos):
    df_final['titulo'] = titulos
else:
    # Si df_final tiene más filas que títulos, eliminar filas adicionales
    df_final = df_final.iloc[:len(titulos)]
    df_final['titulo'] = titulos

# Realizar el left join
df_merged = pd.merge(df1, df_final, on='titulo', how='left')



C:\Users\Usuario\AppData\Local\Temp\ipykernel_9480\1543681339.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['titulo'] = titulos


In [13]:
# Mostrar el DataFrame resultante
df_merged

,peliculas,titulo,Año,Duración,Valoración
0,1. Cadena perpetua,Cadena perpetua,1994,2h 22m,13
1,2. El padrino,El padrino,1972,2h 55m,18
2,3. El caballero oscuro,El caballero oscuro,2008,2h 32m,13
3,4. El padrino parte II,El padrino parte II,1974,3h 22m,18
4,5. 12 hombres sin piedad,12 hombres sin piedad,1957,1h 36m,A
...,...,...,...,...,...
245,246. La leyenda del indomable,La leyenda del indomable,2011,NaN,18
246,247. Criadas y señoras,Criadas y señoras,1992,NaN,18
247,248. Sucedió una noche,Sucedió una noche,1984,NaN,14
248,249. Aladdín,Aladdín,NaN,NaN,1967


In [17]:
# Guardar el DataFrame resultante en un archivo Excel
output_file = 'df_merged.xlsx'
df_merged.to_excel(output_file, index=False)